## 필요 라이브러리 호출

In [1]:
## 사용 라이브러리 호출
import pandas as pd
import numpy as np
import random 
from urllib.parse import quote, unquote
from datetime import timedelta

## 모델 사용 라이브러리 
import torch
import torch.nn as nn
import torch.optim as optim

## 모델 학습 사용 라이브러리
from tqdm.notebook import trange
import statistics
from sklearn.metrics import classification_report

## 모델 학습 결과 경로 설정 
import os
os.makedirs('./result', exist_ok=True)

## Cuda 사용 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

## 랜덤 시드 설정
def set_seed(seed_val):
    random.seed(seed_val)
    np.random.seed(seed_val)
    torch.manual_seed(seed_val)
    torch.cuda.manual_seed_all(seed_val)

# 시드 설정 
seed_val = 77
set_seed(seed_val)

cuda


## 사용 데이터 컬럼 선택

* tag name 이 많은 경우 tag name을 지정하는 것에 있어서 변수 설정이 다소 유연해짐
* tag name 은 순서대로 불러와짐 

In [2]:
# tag name 출력 함수 
def show_column(URL):
    
    # Tag name 데이터 로드
    df = pd.read_csv(URL)
    
    # List 형식으로 변환
    df = df.values.reshape(-1)
    
    return df.tolist()

In [3]:
## tag name 출력 파라미터 설정
table = 'home'

NAME_URL = f'http://127.0.0.1:5654/db/tql/datahub/api/v1/get_tag_names.tql?table={table}'

## tag name list 생성 
name = show_column(NAME_URL)

In [4]:
name

['TAG-Barn [kW]',
 'TAG-Dishwasher [kW]',
 'TAG-Fridge [kW]',
 'TAG-Furnace 1 [kW]',
 'TAG-Furnace 2 [kW]',
 'TAG-Garage door [kW]',
 'TAG-Home office [kW]',
 'TAG-House overall [kW]',
 'TAG-Kitchen 12 [kW]',
 'TAG-Kitchen 14 [kW]',
 'TAG-Kitchen 38 [kW]',
 'TAG-Living room [kW]',
 'TAG-Microwave [kW]',
 'TAG-Solar [kW]',
 'TAG-Well [kW]',
 'TAG-Wine cellar [kW]',
 'TAG-apparentTemperature',
 'TAG-dewPoint',
 'TAG-gen [kW]',
 'TAG-humidity',
 'TAG-precipIntensity',
 'TAG-precipProbability',
 'TAG-pressure',
 'TAG-temperature',
 'TAG-use [kW]',
 'TAG-visibility',
 'TAG-windBearing',
 'TAG-windSpeed']

## TAG Name format 변환 

* 위의 과정에서 Smart home dataset의 모든 Tag Name 을 확인후 사용할 컬럼만 뽑아서 입력할 파라미터 형태로 변환

* TAG-windBearing, TAG-windSpeed Tag Name 사용하여 예제 진행 

In [5]:
# 원하는 tag name 설정
# 여기서 tag name 은 컬럼을 의미
tags = name[-2:]

# 리스트의 각 항목을 작은따옴표로 감싸고, 쉼표로 구분
tags_ = ",".join(f"'{tag}'" for tag in tags)

# 사용 tag name 확인
print(tags_)

# 해당 값을 모델의 input shape로 설정 
print(len(tags))

'TAG-windBearing','TAG-windSpeed'
2


## Smart Home Dataset 로드

* 데이터 로드는 필요할때 마다 필요한 만큼만 로드하여 사용

In [6]:
# 데이터 로드 함수
# '1D': 일간 간격 (1일)
# '1H': 시간 간격 (1시간)
# '1T' 또는 'min': 분 간격 (1분)
# '1S': 초 간격 (1초)
def data_load(able, name, start_time_, end_time_, timeformat, resample_time):
    
    # 데이터 로드 
    df = pd.read_csv(f'http://127.0.0.1:5654/db/tql/datahub/api/v1/select-rawdata.tql?table={table}&name={name}&start={start_time_}&end={end_time_}&timeformat={timeformat}')
    
    # 같은 시간대 별 데이터로 전환
    df = df.pivot_table(index='TIME', columns='NAME', values='VALUE', aggfunc='first').reset_index()
    
    # time index 설정
    df = df.set_index(pd.to_datetime(df['TIME']))
    df = df.drop(['TIME'], axis=1)
    
    # 1초간격 resampling
    # 원하는 간격으로 수정 가능 일, 시, 분 등등 
    df = df.resample(f'{resample_time}').mean()
    
    return df

In [7]:
# 시간 변환 함수
# 시간 추가하려면 해당 과정 필요
# window_size : 데이터를 묶어서 수집되는 주기 
# step_size : 데이터의 간격 
def add_time(start_time, batch_size, window_size, step_size):
    
    # 타임 스탬프 형식으로 변환 
    start_time = pd.Timestamp(start_time)
    
    # 설정한 시간만큼의 시간을 추가
    end_time_ = start_time + timedelta(seconds=(batch_size * step_size)+ window_size - step_size - 1)
    next_start_time_ = end_time_ + timedelta(seconds= -abs(window_size - step_size - 1))
    
    # 다시 원래 형태로 출력
    start_time_ = start_time.strftime('%Y-%m-%d %H:%M:%S')
    end_time_ = end_time_.strftime('%Y-%m-%d %H:%M:%S')
    next_start_time_ = next_start_time_.strftime('%Y-%m-%d %H:%M:%S')
    
    # URL 인코딩
    start_time_ = quote(start_time_)
    end_time_ = quote(end_time_)
    next_start_time_ = quote(next_start_time_)
    
    return start_time_, end_time_, next_start_time_

In [8]:
# 선택한 tag name 별 최대, 최소값 계산 함수
def set_minmax_value(table, name, start_time_train, end_time_train):
    
    # URL 인코딩
    start = quote(start_time_train)
    end = quote(end_time_train)
    
    # min max 데이터 로드
    df_ = pd.read_csv(f'http://127.0.0.1:5654/db/tql/datahub/api/v1/select-scale.tql?table={table}&name={name}&start={start}&end={end}')
    
    ## Min , Max values 설정 
    Min = df_.iloc[:,1:-1].T
    Max = df_.iloc[:,2:].T
    
    return Min, Max 

In [9]:
def make_window(data, window_size, step_size):
    
    # 윈도우 슬라이딩 결과를 저장할 리스트
    windows = []

    # 슬라이딩 윈도우 적용
    for i in range(0, data.shape[0] - window_size + 1, step_size):
        window = data[i:i + window_size, :]
        windows.append(window)
        
    return windows    

## Min-Max Scaling 설정

* Process 컨셉상 전체 데이터를 불러오지 않기 때문에 최대 최소값을 사용하는 방식의 Min-Max Sclare를 설정 

In [10]:
# MinMaxScaler 클래스 정의
class MinMaxScaler:
    def __init__(self):
        self.min_ = None
        self.max_ = None

    # 설정 파라미터 기반 scale 값 설정 
    def transform(self, X, min_values, max_values):
        X = np.array(X)
        self.min_ = np.array(min_values)
        self.max_ = np.array(max_values)
        
        if self.min_ is None or self.max_ is None:
            raise ValueError("Min and Max values are not set.")
        
        # scale 값이 0이되는 것을 막기 위해 1e-6 을 더해줌 
        scale = (self.max_ - self.min_) + 1e-6
        if np.any(scale == 0):
            raise ValueError("Min and Max values are the same, resulting in a scale of 0.")
        
        return (X - self.min_) / scale
    
    # 계산한 scale 값 기준 데이터 정규화 
    def fit_transform(self, X, min_values, max_values):
        """Set parameters and then transform X"""
        return self.transform(X,min_values, max_values)

## 학습 모델 설정 

* LSTM AE 기본 모델 사용

In [11]:
# LSTM Autoencoder 클래스 정의
class LSTMAutoencoder(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers):
        super(LSTMAutoencoder, self).__init__()
        
        # 인코더 LSTM
        self.encoder_lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.encoder_fc = nn.Linear(hidden_dim, 2*hidden_dim)
        
        # 디코더 LSTM
        self.decoder_fc = nn.Linear(2*hidden_dim, hidden_dim)
        self.decoder_lstm = nn.LSTM(hidden_dim, input_dim, num_layers, batch_first=True)

    def forward(self, x):
        # 인코더 부분
        _, (h, _) = self.encoder_lstm(x)
        latent = self.encoder_fc(h[-1])
        
        # 디코더 부분
        hidden = self.decoder_fc(latent).unsqueeze(0).repeat(x.size(1), 1, 1).permute(1, 0, 2)
        output, _ = self.decoder_lstm(hidden)
        
        return output

In [12]:
# 모델 설정 파라미터

# 입력 데이터 컬럼 수
# Tag Name 수와 동일 
input_dim = len(tags)

# LSMT hidden state 크기
hidden_dim = 2*len(tags)

# layer 수
num_layers = 3

# 학습률 
learning_rate = 0.01

# 모델 초기화
model = LSTMAutoencoder(input_dim, hidden_dim, num_layers).to(device)

# 손실 함수 및 옵티마이저 설정
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# 모델 구조 확인
print(model)

LSTMAutoencoder(
  (encoder_lstm): LSTM(2, 4, num_layers=3, batch_first=True)
  (encoder_fc): Linear(in_features=4, out_features=8, bias=True)
  (decoder_fc): Linear(in_features=8, out_features=4, bias=True)
  (decoder_lstm): LSTM(4, 2, num_layers=3, batch_first=True)
)


## 모델 학습

* 필요한 배치 크기의 데이터만 불러와서 학습하는 방법으로 진행

In [13]:
# 모델 학습 함수 설정
def train(table, name, timeformat, model, start_time_train, end_time_train, batch_size, window_size, step_size, epochs):
    
    # 초기 train loss 설정
    train_loss = []
    
    # 베스트 loss 초기화 
    best_Loss=np.inf
    
    # 최대, 최소값 설정 
    Min, Max = set_minmax_value(table, name, start_time_train, end_time_train)
    
    # Min-Max scaler 설정 
    scaler = MinMaxScaler()

    for epoch in epochs:
        
        # 학습 모드 설정 
        model.train()
        
        # 초기 loss 초기화
        running_loss = 0.0
        total_step = 0
        
        # 초기 시작 시간 설정
        start_time_ = start_time_train

        # while 문을 통해 데이터 호출 
        while start_time_ < end_time_train:
            
            # 배치 크기에 따라 데이터 로드 
            start_time_, end_time_, next_start_time_= add_time(start_time_, batch_size, window_size, step_size)
            
            # 데이터 로드 
            data = data_load(table, name, start_time_, end_time_, timeformat, resample_time="1s")
             
            # Min Max Scaling 적용
            scaled_data = scaler.fit_transform(data, Min, Max)
            
            # window 설정 
            windows = make_window(scaled_data, window_size, step_size)
            
            # 로드한 데이터가 비어 있을 경우 출력 
            if len(scaled_data) == 0:
                print("데이터가 없습니다.")
            
            # 배치 사이즈 만큼 데이터가 쌓이면 다음 배치로 이동
            if len(windows) == batch_size:
                
                # 총 배치수 체크용  
                total_step = total_step + 1
                
                # 데이터를 numpy 배열로 변환
                input_data = np.array(windows)

                # 데이터를 Tensor로 변환
                input_data = torch.tensor(input_data, dtype=torch.float32).to(device).float()
 
                # 옵티마이저 최적화 
                optimizer.zero_grad()
                
                # 모델 입력
                outputs = model(input_data)
                
                # loss 계산
                loss = criterion(outputs, input_data)
                loss.backward()
                optimizer.step()
                running_loss += loss.item()
                
                # 배치 리셋
                windows = []
            
            # 다음 시작 시간 설정    
            start_time_ = unquote(next_start_time_)
            
        if total_step > 0:
            train_loss.append(running_loss / total_step)
            print(f'\ntrain loss: {np.mean(train_loss)}')
            
        if best_Loss > np.mean(train_loss):
            best_Loss = np.mean(train_loss)
            torch.save(model, f'./result/Smart_home_LSTM_AE_New_Batch.pt')
            print('베스트 모델 저장') 
            
        epochs.set_postfix_str(f"epoch = {epoch}, best_Loss = {best_Loss}")
               
    return model

In [14]:
## 학습 파라미터 설정
# tag table 이름 설정
table = 'home'
# tag name 설정
name = quote(tags_, safe=":/")
# 시작 시간 설정
start_time_train = '2016-01-01 14:00:00'
# 끝 시간 설정
end_time_train = '2016-01-01 15:00:00'
# 시간 포멧 설정 
timeformat = 'Default'
# 배치 사이즈 설정
batch_size = 32
# window size 설정
window_size = 3
# step size 설정 -> window size 보다 크거나 같으면 안됨 
step_size = 1
# epoch 설정
epochs = trange(100, desc='training')

# 학습 진행
train(table, name, timeformat, model, start_time_train, end_time_train, batch_size, window_size, step_size, epochs)

training:   0%|          | 0/100 [00:00<?, ?it/s]


train loss: 0.12431900912022169
베스트 모델 저장

train loss: 0.07182825464277273
베스트 모델 저장

train loss: 0.05192385768898348
베스트 모델 저장

train loss: 0.04170641856144679
베스트 모델 저장

train loss: 0.035519949847006085
베스트 모델 저장

train loss: 0.03137902159952187
베스트 모델 저장

train loss: 0.028413706540860652
베스트 모델 저장

train loss: 0.02618540141644447
베스트 모델 저장

train loss: 0.024449429648893205
베스트 모델 저장

train loss: 0.023058679470763056
베스트 모델 저장

train loss: 0.02191939610247171
베스트 모델 저장

train loss: 0.020968983457367085
베스트 모델 저장

train loss: 0.020164045368483225
베스트 모델 저장

train loss: 0.01947354743091299
베스트 모델 저장

train loss: 0.01887470757924254
베스트 모델 저장

train loss: 0.01835042426312539
베스트 모델 저장

train loss: 0.017887609563693067
베스트 모델 저장

train loss: 0.01747607563649425
베스트 모델 저장

train loss: 0.01710777228138994
베스트 모델 저장

train loss: 0.016776251792308898
베스트 모델 저장

train loss: 0.01647628772628962
베스트 모델 저장

train loss: 0.016203596700617967
베스트 모델 저장

train loss: 0.015954634201971896
베스트 모델 저장



LSTMAutoencoder(
  (encoder_lstm): LSTM(2, 4, num_layers=3, batch_first=True)
  (encoder_fc): Linear(in_features=4, out_features=8, bias=True)
  (decoder_fc): Linear(in_features=8, out_features=4, bias=True)
  (decoder_lstm): LSTM(4, 2, num_layers=3, batch_first=True)
)

## 임계값 설정

* validation data를 사용하여 임계값 계산 
1) 평균 + 편차 
2) Max 값
3) 99% - std 값

In [15]:
def threshold_set(table, name, timeformat, model, start_time_train, end_time_train, start_time_val, end_time_val, batch_size, window_size, step_size, option):
    valid_loss = []
    with torch.no_grad():
        
        # 초기 시작 시간 설정
        start_time_ = start_time_val
        
        # 최대, 최소값 확인 
        Min, Max = set_minmax_value(table, name, start_time_train, end_time_train)
        
        # Min-Max scaler 설정 
        scaler = MinMaxScaler()
        
        # while 문을 통해 데이터 호출 
        while start_time_ < end_time_val:
            
            # 윈도우 슬라이딩에 따라 데이터 
            start_time_, end_time_, next_start_time_= add_time(start_time_, batch_size,window_size, step_size)
            
            # 데이터 로드 
            data = data_load(table, name, start_time_, end_time_, timeformat, resample_time="1s")
            
            # Min Max Scaling 적용
            scaled_data = scaler.fit_transform(data, Min, Max)
            
            # window 설정 
            windows = make_window(scaled_data, window_size, step_size)
            
            # 로드한 데이터가 비어 있을 경우 출력 
            if len(scaled_data) == 0:
                print("데이터가 없습니다.")
            
            # 배치 사이즈 만큼 데이터가 쌓이면 다음 배치로 이동
            if len(windows) == batch_size:
                
                # 데이터를 numpy 배열로 변환
                input_data_val = np.array(windows)
                
                # 데이터를 Tensor로 변환
                input_data_val = torch.tensor(input_data_val, dtype=torch.float32).to(device).float()

                # 모델 입력
                outputs_val = model(input_data_val)
                
                # loss 계산
                loss_val = criterion(outputs_val, input_data_val)
            
                valid_loss.append(loss_val.item())
                
                # 배치 리셋
                windows = []
                
            # 다음 시작 시간 설정    
            start_time_ = unquote(next_start_time_)
            
        # 임계값 계산
        if option == 0:
            # 평균 + 편차값 
            threshold =  statistics.mean(valid_loss) + statistics.stdev(valid_loss) 
            
        # 임계값 계산
        if option == 1:
            # MAX 값 
            threshold =  max(valid_loss)
             
        # 임계값 계산
        if option == 2:
            # 99% - std 값  
            threshold =  np.percentile(valid_loss, 99) - statistics.stdev(valid_loss)             
                    
    return threshold

In [50]:
# 검증 데이터 범위 설정 
start_time_val = '2016-01-01 15:00:00'
end_time_val = '2016-01-01 16:00:00'

# 베스트 모델 로드
model = torch.load(f'./result/Smart_home_LSTM_AE_New_Batch.pt')

# 임계값 옵션 설정
option = 2

# 임계값 설정
threshold = threshold_set(table, name, timeformat, model, start_time_train, end_time_train, start_time_val, end_time_val, batch_size, window_size, step_size, option)

## 모델 테스트

* 이전 단계에서 계산한 임계값을 기준으로 테스트 데이터를 통한 모델 테스트 진행

In [51]:
## 테스트 함수 
def test(table, name, timeformat, model, start_time_train, end_time_train, start_time_test, end_time_test, batch_size, window_size, step_size, threshold):
    test_loss = []
    with torch.no_grad():
        
        # 초기 시작 시간 설정
        start_time_ = start_time_test
        
        # 최대, 최소 값 설정 
        Min, Max = set_minmax_value(table, name, start_time_train, end_time_train)

        # Min-Max scaler 설정 
        scaler = MinMaxScaler()
    
        # while 문을 통해 데이터 호출 
        while start_time_ < end_time_test:
            
            # 윈도우 슬라이딩에 따라 데이터 
            start_time_, end_time_, next_start_time_= add_time(start_time_, batch_size, window_size, step_size)
    
            # 데이터 로드 
            data = data_load(table, name, start_time_, end_time_, timeformat, resample_time="1s")
            
            # Min Max Scaling 적용
            scaled_data = scaler.fit_transform(data, Min, Max)
            
            # window 설정 
            windows = make_window(scaled_data, window_size, step_size)
            
            # 로드한 데이터가 비어 있을 경우 출력 
            if len(scaled_data) == 0:
                print("데이터가 없습니다.")
            
            # 배치 사이즈 만큼 데이터가 쌓이면 다음 배치로 이동
            if len(windows) == batch_size:
                
                # 데이터를 numpy 배열로 변환
                input_data_test = np.array(windows)
                
                # 데이터를 Tensor로 변환
                input_data_test = torch.tensor(input_data_test, dtype=torch.float32).to(device).float()

                # 모델 입력
                outputs_test = model(input_data_test)
                
                # loss 계산
                loss_test = criterion(outputs_test, input_data_test)
            
                test_loss.append(loss_test.item())
                
                # 배치 리셋
                windows = []
                
            # 다음 시작 시간 설정    
            start_time_ = unquote(next_start_time_) 
            
    # 최종 결과 생성 
    final_df = pd.DataFrame(test_loss, columns=['reconst_score'])
    final_df['label'] = 0

    # 각 임계값 별 label 설정 
    final_df['pred_label'] = np.where(final_df['reconst_score']>threshold,1,0)
    
    return final_df

In [52]:
# 테스트 데이터 범위 설정 
start_time_test = '2016-01-01 16:00:00'
end_time_test = '2016-01-01 17:00:00'

result_df = test(table, name, timeformat, model, start_time_train, end_time_train, start_time_test, end_time_test, batch_size, window_size, step_size, threshold)

# 성능 평가

* F1 Score 기준으로 평가 진행
* 옵션별로 성능 평가 진행  

In [45]:
# 0. 평균 + 편차 임계값 설정
print(f'임계값 {threshold}') 
print(classification_report(result_df['label'], result_df['pred_label'],labels=[0]))

임계값 0.12877620475788487
              precision    recall  f1-score   support

           0       1.00      0.43      0.60       113

   micro avg       1.00      0.43      0.60       113
   macro avg       1.00      0.43      0.60       113
weighted avg       1.00      0.43      0.60       113



In [49]:
# 1. Max 값
print(f'임계값 {threshold}') 
print(classification_report(result_df['label'], result_df['pred_label'],labels=[0]))

임계값 0.3305935859680176
              precision    recall  f1-score   support

           0       1.00      0.99      1.00       113

   micro avg       1.00      0.99      1.00       113
   macro avg       1.00      0.99      1.00       113
weighted avg       1.00      0.99      1.00       113



In [53]:
# 2. 99% - std 값
print(f'임계값 {threshold}') 
print(classification_report(result_df['label'], result_df['pred_label'],labels=[0]))

임계값 0.20730006255657174
              precision    recall  f1-score   support

           0       1.00      0.73      0.84       113

   micro avg       1.00      0.73      0.84       113
   macro avg       1.00      0.73      0.84       113
weighted avg       1.00      0.73      0.84       113

